In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
#from useful_functions import *
import csv
from astropy.io import votable
from astropy.table import Table

In [3]:
#Read CSC 2.1 into dataframe
data=pd.read_csv('/Users/kciurleo/Documents/kciurleo/AGN/csvs/CSC2.1p_OIR_SDSSspecmatch.csv')

/var/folders/6x/hz5y30457mqg1y9dl2fg7r5m00034d/T/ipykernel_3919/804397050.py:2: DtypeWarning: Columns (57,61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv('/Users/kciurleo/Documents/kciurleo/AGN/csvs/CSC2.1p_OIR_SDSSspecmatch.csv')


In [4]:
#Read 4XMM-DR13 into dataframe
XMM=pd.read_csv('/Users/kciurleo/Documents/kciurleo/AGN/csvs/4XMM_DR13cat_v1.0.csv')

In [5]:
#Read in SPIDERS data
SPIDERSROS = votable.parse_single_table('/Users/kciurleo/Documents/kciurleo/AGN/csvs/spidersros.xml').to_table().to_pandas()
SPIDERSXMM = votable.parse_single_table('/Users/kciurleo/Documents/kciurleo/AGN/csvs/spidersxmm.xml').to_table().to_pandas()

In [6]:
#Rename XMM ra/dec to avoid confusion later on
XMM['XMM_ra']=XMM['ra']
XMM['XMM_dec']=XMM['dec']

#Add IAU name for easy crossmatching later
XMM['IAU_stripped']=XMM['iauname'].str[5:]

In [7]:
#Find only sources with SDSS data
sources = data.dropna(subset=['Sep_SPEC_CSC21P'])

#Find only non-extended CSC sources
point_sources = sources.loc[sources['extent_flag']==False]

In [8]:
#Saved point sources as csv, which was used in SciServer CasJobs SQL query to get Portsmouth classifications
point_sources.to_csv('/Users/kciurleo/Documents/kciurleo/AGN/csvs/point_sources.csv', index=False) 

In [9]:
#Portsmouth classifications https://salims.pages.iu.edu/agn/
portsmouth=pd.read_csv('/Users/kciurleo/Documents/kciurleo/AGN/csvs/point_sources_classified_lines.csv')

#Agostino classifications https://salims.pages.iu.edu/agn/
agostino=pd.read_csv('/Users/kciurleo/Documents/kciurleo/AGN/csvs/agostino2021_table1.csv')

In [10]:
#Find all XMM SPIDERS 
xspi_s2 = SPIDERSXMM.loc[(SPIDERSXMM['source_type']=="NLAGN")]

#Find all ROSAT SPIDERS which are point sources, and then which are S2
rosspi_points = SPIDERSROS.loc[(SPIDERSROS['extent_likelihood']<1)]
rosspi_s2 = rosspi_points.loc[(rosspi_points['source_type']=="NLAGN")]

#Combined SPIDERS s2
spi_s2 = pd.merge(rosspi_s2, xspi_s2, how="outer", on=['sdss_spec_plate_num', 'sdss_spec_mjd_num', 'sdss_spec_fiber_num'])

In [11]:
#Get the agostino spectral ids and merge into normal agostino table
agostino_IDs=pd.read_csv('/Users/kciurleo/Documents/kciurleo/AGN/csvs/agostino_specIDs.csv')
agostino_full =pd.merge(agostino_IDs, agostino, left_on=['objID'], right_on=['SDSS_ObjID'], how='inner')

#Combine our point source table, portsmouth classification, and agostino classifications
combined = pd.merge(point_sources, portsmouth, left_on=['PLATE', 'MJD', 'FIBERID'],right_on=['plate', 'mjd', 'fiberID'], how='left')
classified_point_sources = pd.merge(combined, agostino_full, left_on =['specobjID'],right_on=['specobjID'],how='left')

#Add IAU name to our sources
classified_point_sources['IAU_stripped']=classified_point_sources['CSC21P_name'].str[5:]

In [12]:
#Make full table of XMM and CSC data
full_point_sources=pd.merge(classified_point_sources,XMM,how='left', on=['IAU_stripped'])
full_point_sources.to_csv('/Users/kciurleo/Documents/kciurleo/AGN/csvs/full_point_sources.csv', index=False) 

#Subset of point sources with XMM data
XMM_point_sources=full_point_sources.loc[full_point_sources['detid'] >0]

In [13]:
#Find only Seyfert Galaxies, classified as bpt="Seyfert" for Portsmouth and sl_class1=1 for Agostino (latter is specifically Seyfert 2s)
portsmouth_s2=full_point_sources.loc[full_point_sources['bpt']=="Seyfert"]
agostino_s2=full_point_sources.loc[full_point_sources['sl_class1']==1]

#Those classified by both
inner_s2=pd.merge(agostino_s2, portsmouth_s2, how='inner')

#Those classified by either
outer_s2=pd.merge(agostino_s2, portsmouth_s2, how='outer')

#Those that are fully unclassified by both agostino and portsmouth
unclassified = full_point_sources.loc[((full_point_sources.bpt.isnull()) | (full_point_sources['bpt']=="BLANK")) & ((full_point_sources['sl_class1']==0) | (full_point_sources.sl_class1.isnull()))] 

In [61]:
print(f'Total observations in crossmatch: {len(data):,}')
print(f'Sources matched with SDSS: {len(sources):,}; {len(data)-len(sources):,} not observed with SDSS')
print(f'Point sources: {len(point_sources):,}; {len(sources)-len(point_sources):,} extended sources')
print(f'Unique point sources with exact XMM data: {len(XMM_point_sources["IAU_stripped"].unique()):,}')
print()
print(f'Unique SPIDERSROS point sources: {len(rosspi_points["name"].unique()):,}; {len(SPIDERSROS["name"].unique()) - len(rosspi_points["name"].unique()):,} sources with extent likelihood >1')
print(f'Unique SPIDERSROS NLAGN: {len(rosspi_s2["name"].unique()):,}; {len(rosspi_points["name"].unique())-len(rosspi_s2["name"].unique()):,} non-Seyfert 2s')
print(f'Unique SPIDERSXMM NLAGN: {len(xspi_s2["name"].unique()):,}; {len(SPIDERSXMM["name"].unique())-len(xspi_s2["name"].unique()):,} non-Seyfert 2s')
print(f'Total SPIDERS NLAGN: {len(spi_s2):,}')
print(f"Total SPIDERS present in point sources of CSC2.1: {len(pd.merge(spi_s2, classified_point_sources, left_on=['sdss_spec_plate_num', 'sdss_spec_mjd_num', 'sdss_spec_fiber_num'], right_on=['PLATE', 'MJD', 'FIBERID'],how='inner')):,}")
print(f"Total SPIDERS present in Portsmouth or Agostino Seyferts: {len(pd.merge(spi_s2, outer_s2, left_on=['sdss_spec_plate_num', 'sdss_spec_mjd_num', 'sdss_spec_fiber_num'], right_on=['PLATE', 'MJD', 'FIBERID'],how='inner')):,}")
print()
print(f'Unique unclassifiable or non-classified sources: {len(unclassified["CSC21P_name"].unique()):,}')
print(f'Unique Portsmouth Seyferts: {len(portsmouth_s2["CSC21P_name"].unique()):,}; {len(point_sources["CSC21P_name"].unique())-len(unclassified["CSC21P_name"].unique())-len(portsmouth_s2["CSC21P_name"].unique()):,} classified non-Seyferts')
print(f'Unique Agostino Seyferts: {len(agostino_s2["CSC21P_name"].unique()):,}; {len(point_sources["CSC21P_name"].unique())-len(unclassified["CSC21P_name"].unique())-len(agostino_s2["CSC21P_name"].unique()):,} classified non-Seyferts')
print()
print(f'Unique Portsmouth-Agostino Seyferts: {len(inner_s2["CSC21P_name"].unique()):,}')
print(f'Unique Portsmouth or Agostino Seyferts: {len(outer_s2["CSC21P_name"].unique()):,}')
print(f"Unique Portsmouth or Agostino Seyferts processed with 2.1: {len(outer_s2.loc[outer_s2['csc2.1_flag']==True]['CSC21P_name'].unique()):,}; {len(outer_s2) -len(outer_s2.loc[outer_s2['csc2.1_flag']==True]['CSC21P_name'].unique()):,} not yet processed")
print(f'Unique Portsmouth or Agostino Seyferts with exact XMM data: {len(outer_s2["detid"].unique()):,}')

Total observations in crossmatch: 387,441
Sources matched with SDSS: 17,666; 369,775 not observed with SDSS
Point sources: 16,022; 1,644 extended sources
Unique point sources with exact XMM data: 1,146

Unique SPIDERSROS point sources: 19,515; 1,773 sources with extent likelihood >1
Unique SPIDERSROS NLAGN: 177; 19,338 non-Seyfert 2s
Unique SPIDERSXMM NLAGN: 21; 2,635 non-Seyfert 2s
Total SPIDERS NLAGN: 185
Total SPIDERS present in point sources of CSC2.1: 5
Total SPIDERS present in Portsmouth or Agostino Seyferts: 0

Unique unclassifiable or non-classified sources: 12,903
Unique Portsmouth Seyferts: 660; 2,459 classified non-Seyferts
Unique Agostino Seyferts: 130; 2,989 classified non-Seyferts

Unique Portsmouth-Agostino Seyferts: 73
Unique Portsmouth or Agostino Seyferts: 717
Unique Portsmouth or Agostino Seyferts processed with 2.1: 667; 158 not yet processed
Unique Portsmouth or Agostino Seyferts with exact XMM data: 157


In [16]:
#Read in 2.0
old_data=pd.read_csv('/Users/kciurleo/Documents/kciurleo/AGN/csvs/oldXmatch.csv')

In [17]:
#To compare CSC2.1 to CSC2.0:
both_data = data.merge(old_data, indicator=True, how='inner', left_on='CSC21P_name', right_on='CSC2_ID')

In [18]:
print(f'Sources in 2.1: {len(data):,}')
print(f'Sources in 2.0: {len(old_data):,}')
print(f'Sources in both: {len(both_data):,}')
print(f'Sources from 2.0 excluded from 2.1: {len(old_data)-len(both_data):,} ')
print(f'New sources in 2.1: {len(data)-len(both_data):,}')

Sources in 2.1: 387,441
Sources in 2.0: 35,771
Sources in both: 35,553
Sources from 2.0 excluded from 2.1: 218 
New sources in 2.1: 351,888


In [19]:
#Seth's list of unique unabsorbed candidates(?), not sure where in the pipeline this comes from
colnames=['CXOname', 'OBSID', 'specObjID', 'RA', 'DEC', 'Z', 'nH', 'Counts'] 
seth_list = pd.read_csv('/Users/kciurleo/Documents/kciurleo/AGN/csvs/source_info_obsid_unique_seth.txt', names=colnames, comment='#',sep="  ")
tester_table=pd.merge(classified_point_sources, seth_list, how='inner', left_on='CSC21P_name',right_on='CXOname')

/var/folders/6x/hz5y30457mqg1y9dl2fg7r5m00034d/T/ipykernel_3919/3576948673.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  seth_list = pd.read_csv('/Users/kciurleo/Documents/kciurleo/AGN/csvs/source_info_obsid_unique_seth.txt', names=colnames, comment='#',sep="  ")


In [20]:
seth_list.to_csv('/Volumes/galaxies/rerunning_seth/source_info_obsid_unique_seth.csv')

In [21]:
haves = set(pd.read_csv("/Users/kciurleo/Documents/kciurleo/AGN/csvs/haves.txt", header=None)[0])

In [22]:
def strip_letters(id_str):
    return ''.join(filter(str.isdigit, id_str))

# Apply function and convert to numeric
seth_list['OBSID'] = seth_list['OBSID'].apply(strip_letters)
seth_list['OBSID'] = pd.to_numeric(seth_list['OBSID'])

In [23]:
seth_list.to_csv('/Volumes/galaxies/rerunning_seth/source_info_obsid_unique_seth.csv', index=False)

In [24]:
neededIDs = set(seth_list['OBSID'])-set(haves)

In [25]:
#

with open('seths_obsids.txt', 'w') as f:
    for line in neededIDs:
        f.write(f"{line}\n")


In [27]:
print(f"Seth's unabsorbed list: {len(seth_list)}")
print(f"On both lists: {len(tester_table)}")
print(f"On my list: {len(classified_point_sources)}, but still need to weed out wavdetect/below 50 ct sources/unabsorbed")

Seth's unabsorbed list: 323
On both lists: 281
On my list: 16022, but still need to weed out wavdetect/below 50 ct sources/unabsorbed


In [28]:
XMM_outer_s2=outer_s2.loc[outer_s2['detid'] >0]

In [29]:
seth_true_list=["J235720.1-005829",
"J104429.3+113810",
"J115930.3+532055",
"J143232.4+340624",
"J080534.7+240951",
"J125830.1-015836",
"J135317.9+332929",
"J153854.0+170137",
"J014209.0-005050",
"J134736.4+173404",
"J081145.3+232825",
"J142841.1+323222",
"J115852.2+424320"]

In [30]:
XMM_outer_s2[XMM_outer_s2['IAU_stripped'].isin(seth_true_list)]

,CSC21P_name,ra_x,dec_x,likelihood_class,extent_flag,err_ellipse_r0,err_ellipse_r1,err_ellipse_ang,significance,flux_aper_b,...,sc_ep_8_fmin_err,sc_ep_8_fmax,sc_ep_8_fmax_err,mjd_first,mjd_last,n_detections,confused,high_background,XMM_ra,XMM_dec
559,2CXO J134736.4+173404,206.901667,17.56794,TRUE,False,0.71614,0.715051,151.537133,13.510591,3.393152e-13,...,4.847249e-14,7.282900e-13,6.677797e-14,52814.157106,55946.957558,9.0,f,f,206.901640,17.567958
560,2CXO J134736.4+173404,206.901667,17.56794,TRUE,False,0.71614,0.715051,151.537133,13.510591,3.393152e-13,...,4.847249e-14,7.282900e-13,6.677797e-14,52814.157106,55946.957558,9.0,f,f,206.901976,17.567607
561,2CXO J134736.4+173404,206.901667,17.56794,TRUE,False,0.71614,0.715051,151.537133,13.510591,3.393152e-13,...,4.847249e-14,7.282900e-13,6.677797e-14,52814.157106,55946.957558,9.0,f,f,206.901640,17.567938
562,2CXO J134736.4+173404,206.901667,17.56794,TRUE,False,0.71614,0.715051,151.537133,13.510591,3.393152e-13,...,4.847249e-14,7.282900e-13,6.677797e-14,52814.157106,55946.957558,9.0,f,f,206.902272,17.567583
563,2CXO J134736.4+173404,206.901667,17.56794,TRUE,False,0.71614,0.715051,151.537133,13.510591,3.393152e-13,...,4.847249e-14,7.282900e-13,6.677797e-14,52814.157106,55946.957558,9.0,f,f,206.902066,17.567721
564,2CXO J134736.4+173404,206.901667,17.56794,TRUE,False,0.71614,0.715051,151.537133,13.510591,3.393152e-13,...,4.847249e-14,7.282900e-13,6.677797e-14,52814.157106,55946.957558,9.0,f,f,206.901044,17.568246
565,2CXO J134736.4+173404,206.901667,17.56794,TRUE,False,0.71614,0.715051,151.537133,13.510591,3.393152e-13,...,4.847249e-14,7.282900e-13,6.677797e-14,52814.157106,55946.957558,9.0,f,f,206.901051,17.567927
566,2CXO J134736.4+173404,206.901667,17.56794,TRUE,False,0.71614,0.715051,151.537133,13.510591,3.393152e-13,...,4.847249e-14,7.282900e-13,6.677797e-14,52814.157106,55946.957558,9.0,f,f,206.901791,17.567733
567,2CXO J134736.4+173404,206.901667,17.56794,TRUE,False,0.71614,0.715051,151.537133,13.510591,3.393152e-13,...,4.847249e-14,7.282900e-13,6.677797e-14,52814.157106,55946.957558,9.0,f,f,206.902036,17.567864


In [31]:
classified_point_sources[classified_point_sources['IAU_stripped'].isin(seth_true_list)]

,CSC21P_name,ra,dec,likelihood_class,extent_flag,err_ellipse_r0,err_ellipse_r1,err_ellipse_ang,significance,flux_aper_b,...,Flux_SII_6730,Flux_SII_6730_Err,Flux_OI_6363,Flux_OI_6363_Err,objID,SDSS_ObjID,gen_el_class,sl_class1,sl_class2,IAU_stripped
1219,2CXO J014209.0-005050,25.537575,-0.847252,TRUE,False,0.710904,0.710565,82.134537,23.525578,3.043410e-13,...,98.044530,16.736120,5.179852,6.785204,1.237657e+18,1.237657e+18,5.0,1.0,1.0,J014209.0-005050
2835,2CXO J081145.3+232825,122.938814,23.473876,TRUE,False,0.772866,0.761060,105.354098,2.702703,3.231904e-15,...,64.652080,4.943286,14.526220,4.463283,NaN,NaN,NaN,NaN,NaN,J081145.3+232825
6236,2CXO J104429.3+113810,161.122114,11.636376,TRUE,False,2.965642,2.072851,91.362308,3.333333,2.197963e-14,...,44.010520,12.255660,8.405495,7.712914,1.237661e+18,1.237661e+18,5.0,1.0,1.0,J104429.3+113810
8139,2CXO J115930.3+532055,179.876264,53.348816,TRUE,False,0.736751,0.731579,98.703095,3.371429,2.698659e-14,...,125.072200,11.233980,78.419590,23.759050,NaN,NaN,NaN,NaN,NaN,J115930.3+532055
9862,2CXO J125830.1-015836,194.625769,-1.976809,MARGINAL,False,6.912127,3.616240,42.532975,3.636364,0.000000e+00,...,415.863300,19.779880,47.190610,11.460640,NaN,NaN,NaN,NaN,NaN,J125830.1-015836
10835,2CXO J134736.4+173404,206.901667,17.567940,TRUE,False,0.716140,0.715051,151.537133,13.510591,3.393152e-13,...,177.470500,6.991809,45.761100,5.844390,NaN,NaN,NaN,NaN,NaN,J134736.4+173404
10951,2CXO J135317.9+332929,208.324704,33.491413,TRUE,False,0.783603,0.783603,0.000000,4.594595,2.740207e-15,...,1317.272000,65.781450,435.735500,66.931560,NaN,NaN,NaN,NaN,NaN,J135317.9+332929
11978,2CXO J142841.1+323222,217.171263,32.539594,TRUE,False,0.802384,0.774075,24.566207,9.301724,2.195996e-13,...,168.700400,17.501150,71.140850,25.319030,1.237665e+18,1.237665e+18,5.0,1.0,1.0,J142841.1+323222
12244,2CXO J143232.4+340624,218.135410,34.106902,TRUE,False,2.381376,1.459633,15.405739,3.085714,8.354569e-15,...,176.786800,10.451690,24.323910,7.726170,1.237663e+18,1.237663e+18,5.0,1.0,1.0,J143232.4+340624
15983,2CXO J235720.1-005829,359.333916,-0.974908,TRUE,False,0.810391,0.755359,81.777545,16.170926,8.609062e-14,...,9.911661,6.223437,5.155740,5.212724,NaN,NaN,NaN,NaN,NaN,J235720.1-005829


In [32]:
seth_finals_not_in_points=["2CXO J080534.7+240951",
"2CXO J153854.0+170137",
"2CXO J115852.2+424320"]

In [33]:
#Pulled Seth's 1252 total observations, before throwing away those with at least 50 counts, which are 663 unique objects
seth_candidates = pd.read_csv('/Users/kciurleo/Documents/kciurleo/all_info_final.csv')
data_overlap = pd.merge(seth_candidates, data, left_on=['SDSS Plate','SDSS Fiber', 'SDSS MJD'], right_on=['PLATE','FIBERID','MJD'], how='inner')
point_overlap=pd.merge(seth_candidates, point_sources, left_on=['SDSS Plate','SDSS Fiber', 'SDSS MJD'], right_on=['PLATE','FIBERID','MJD'], how='inner')
s2_overlap = pd.merge(seth_candidates, outer_s2, left_on=['SDSS Plate','SDSS Fiber', 'SDSS MJD'], right_on=['PLATE','FIBERID','MJD'], how='inner')

In [34]:
print(f"Seth's unique candidates: {len(seth_candidates['# Name'].unique())}")
print(f"Seth's unique candidates present in CSC2.1: {len(data_overlap['# Name'].unique())}, {len(seth_candidates['# Name'].unique())-len(data_overlap['# Name'].unique())} missing")
print(f"Seth's unique candidates present in CSC2.1 point sources: {len(point_overlap['# Name'].unique())}, {len(data_overlap['# Name'].unique())-len(point_overlap['# Name'].unique())} extended sources")
print(f"Seth's unique candidates present in CSC2.1 outer Seyferts: {len(s2_overlap['# Name'].unique())}, {len(point_overlap['# Name'].unique())-len(s2_overlap['# Name'].unique())} classified as Seyfert/LINER, Composite, or LINER in 2.1")
print()
print(f"Unique non-Seth candidates present in CSC2.1 outer Seyferts: {len(outer_s2['CSC21P_name'].unique())-len(s2_overlap['# Name'].unique())}")
print(f"Total candidates present in CSC2.1: {len(outer_s2['CSC21P_name'].unique())}")

Seth's unique candidates: 663
Seth's unique candidates present in CSC2.1: 559, 104 missing
Seth's unique candidates present in CSC2.1 point sources: 499, 60 extended sources
Seth's unique candidates present in CSC2.1 outer Seyferts: 476, 23 classified as Seyfert/LINER, Composite, or LINER in 2.1

Unique non-Seth candidates present in CSC2.1 outer Seyferts: 241
Total candidates present in CSC2.1: 717


In [35]:
#ones in seth's list and in point sources overlap but not s2 overlap
missing_guys = list(set(point_overlap['# Name']) - set(s2_overlap['# Name']))

#Some of the guys in seth's list are classified differently than mine
print(classified_point_sources[classified_point_sources['CSC21P_name'].isin(missing_guys)]['sl_class1'].unique()) #0 is unclassifiable
print(classified_point_sources[classified_point_sources['CSC21P_name'].isin(missing_guys)]['bpt'].unique())

[ 0. nan]
['Seyfert/LINER' 'Composite' 'LINER']


In [36]:
#Leaving this here for now - code to keep integer values of stuff
tester = pd.DataFrame([[302986678874695680, 'yay'], [302986953752602624,'boo']], columns=['specobjID', 'thing'])
othertester = pd.DataFrame([['yay', 'huh'], ['woah','no']], columns=['thing', 'stuff'])

# Convert specobjID to integer in tester DataFrame
tester['specobjID'] = tester['specobjID'].astype(int)

# Merge the DataFrames while preserving integer data type for specobjID
merged_df = pd.merge(tester, othertester, how='outer')

# Print the merged DataFrame
merged_df['specobjID']

0    3.029870e+17
1             NaN
2    3.029867e+17
Name: specobjID, dtype: float64

In [37]:
import os

In [38]:
np.sum(pd.read_csv('/Users/kciurleo/Documents/kciurleo/AGN/csvs/obsids_seyferts.csv')[' TIME'])

63246.399999999994

In [39]:
#Get total exposure time for unique chandra obsids
uniqueOBSIDS = pd.read_csv('/Users/kciurleo/Documents/kciurleo/AGN/csvs/obsids_seyferts.csv').drop_duplicates(subset='CHANDRA_OBSID')
print(f"{np.sum(uniqueOBSIDS[' TIME'])} ks or {np.sum(uniqueOBSIDS[' TIME'])*1000/(np.pi*10**7)} years")

39940.2 ks or 1.2713360516157837 years


In [40]:
seth_true_list

['J235720.1-005829',
 'J104429.3+113810',
 'J115930.3+532055',
 'J143232.4+340624',
 'J080534.7+240951',
 'J125830.1-015836',
 'J135317.9+332929',
 'J153854.0+170137',
 'J014209.0-005050',
 'J134736.4+173404',
 'J081145.3+232825',
 'J142841.1+323222',
 'J115852.2+424320']

In [41]:
reran_min_abs=pd.read_csv('/Volumes/galaxies/rerunning_seth/data_min_abs/min_abs_info.txt' , delimiter='  ', header=1, names=['Name','Obsid','ra','dec','z','nH', 'Counts'])
reran_min_abs['IAU_stripped']=reran_min_abs['Name'].str[5:]

/var/folders/6x/hz5y30457mqg1y9dl2fg7r5m00034d/T/ipykernel_3919/313756062.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  reran_min_abs=pd.read_csv('/Volumes/galaxies/rerunning_seth/data_min_abs/min_abs_info.txt' , delimiter='  ', header=1, names=['Name','Obsid','ra','dec','z','nH', 'Counts'])


In [42]:
len(reran_min_abs['Name'])

33

In [45]:
reran_min_abs[reran_min_abs['IAU_stripped'].isin(seth_unabsorbed_table51)]

,Name,Obsid,ra,dec,z,nH,Counts,IAU_stripped
15,2CXO J081013.0+345137,21707,122.55423,34.860239,0.083139,0.0418,124,J081013.0+345137
19,2CXO J104429.3+113810,5929,161.12194,11.636486,0.229776,0.0228,86,J104429.3+113810
24,2CXO J150117.0+014958,13246,225.32127,1.832829,0.129253,0.0417,240,J150117.0+014958


In [44]:
seth_unabsorbed_table51=['J142859.9+330318',
'J102025.7+25340',
'J081013.0+345137',
'J142044.9+533913',
'J173801.1+58301',
'J233731.7+002600',
'J123038.9+401614',
'J081937.7+21065',
'J113645.8+070647',
'J235720.1-005829',
'J104429.3+113810',
'J115930.3+532055',
'J143232.4+340624',
'J080534.7+240951',
'J125830.1-015836',
'J135317.9+332929',
'J153854.0+170137',
'J014209.0-005050',
'J134736.4+173404',
'J081145.3+232825',
'J142841.1+323',
'J081910.7+212609',
'J161046.8+522805',
'J115852.2+424320',
'J140052.5-014511',
'J150117.0+014958',
'J020001.7-085540',
'J151215.7+020317',
'J140354.3+542209',
'J143525.3+330520']

In [46]:
len(seth_unabsorbed_table51)

30

In [47]:
candidate_info=pd.read_csv('/Volumes/galaxies/Seth/AGNs/x-ray/final_data/candidate_info_final_all.csv')

In [48]:
candidate_info

,# Name,RA,Dec,ObsID,Counts,Z,Model,Cstat,nH,nH error plus,...,Gemini North best day,Gemini South alt,Gemini South best day,SDSS Plate,SDSS Fiber,SDSS MJD,Chandra Cycle,Chandra Exposure (ks),Count rate (/s),Chandra offset (deg)
0,2CXO J155205.6+200843,238.023560,20.145263,10244,27,0.036282,main,448.750452,0.001798,0.258714,...,2023-05-26 10:00:00.000,39.683458,2023-06-16 03:00:00.000,2172,337,54230,10,35.70,0.000756,0.118831
1,2CXO J081013.0+345137,122.554091,34.860185,10359,359,0.082851,main,662.361544,0.000784,0.023346,...,2024-01-29 10:00:00.000,24.969802,2024-02-19 03:00:00.000,892,58,52378,10,7.85,0.045732,0.122921
2,2CXO J123038.9+401614,187.662315,40.270759,10739,110,0.132453,main,542.677172,0.004464,0.281880,...,2023-04-05 10:00:00.000,19.615924,2023-04-26 03:00:00.000,1986,599,53475,10,4.97,0.022133,0.116206
3,2CXO J233752.7+002421,354.469706,0.405921,11728,21,0.089453,main,541.416682,0.008832,NaN,...,2023-09-21 10:00:00.000,59.220531,2023-10-12 03:00:00.000,385,386,51877,11,16.81,0.001249,0.140204
4,2CXO J233731.7+002600,354.382457,0.433293,11728d,112,0.313821,main,541.416682,0.008832,NaN,...,2023-09-21 10:00:00.000,59.193266,2023-10-12 03:00:00.000,682,493,52525,11,16.81,0.006663,0.161041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,2CXO J143821.8+034012,219.591130,3.670339,7707,118,0.224962,res,802.767174,0.000000,0.000000,...,2023-05-07 10:00:00.000,56.189815,2023-05-28 03:00:00.000,586,151,52023,8,5.11,0.023092,0.000363
174,2CXO J143525.3+330520,218.855260,33.089242,7771,78,0.199547,res,612.534716,0.028311,0.193063,...,2023-05-07 10:00:00.000,26.769986,2023-05-27 03:00:00.000,1841,547,53491,8,14.85,0.005253,0.163245
175,2CXO J120153.5+580340,180.473180,58.061222,7893,60,0.099135,res,427.745441,0.000000,0.000000,...,2023-03-29 10:00:00.000,1.824767,2023-04-19 03:00:00.000,1314,391,53050,8,4.99,0.012024,0.049696
176,2CXO J081937.7+210652,124.907830,21.114303,7929,159,0.018440,res,805.118540,0.028647,0.079821,...,2024-02-01 10:00:00.000,38.717462,2023-02-21 03:00:00.000,1927,261,53321,8,28.32,0.005614,0.099863


In [49]:
reran_min_abs[reran_min_abs['Name'].isin(candidate_info['# Name'])]

,Name,Obsid,ra,dec,z,nH,Counts,IAU_stripped
0,2CXO J143821.8+034012,7707,219.59113,3.670339,0.224962,0.0266,118,J143821.8+034012
3,2CXO J112422.6+213132,19768,171.09416,21.525566,0.189377,0.0137,70,J112422.6+213132
8,2CXO J082657.0+360659,15381b,126.73814,36.116429,0.139580,0.0436,58,J082657.0+360659
15,2CXO J081013.0+345137,21707,122.55423,34.860239,0.083139,0.0418,124,J081013.0+345137
16,2CXO J081937.7+210652,7929,124.90783,21.114303,0.018440,0.0427,159,J081937.7+210652
18,2CXO J135317.7+332927,4867,208.32412,33.490851,0.008187,0.0153,548,J135317.7+332927
19,2CXO J104429.3+113810,5929,161.12194,11.636486,0.229776,0.0228,86,J104429.3+113810
20,2CXO J111246.7+132401,4216,168.19481,13.400363,0.068769,0.0174,293,J111246.7+132401
21,2CXO J090236.8+520349,11588,135.65353,52.063553,0.099548,0.0189,287,J090236.8+520349
24,2CXO J150117.0+014958,13246,225.32127,1.832829,0.129253,0.0417,240,J150117.0+014958


In [50]:
len(candidate_info['# Name'].unique())

94

In [51]:
candidate_info['IAU_stripped']=candidate_info['# Name'].str[5:]

In [52]:
candidate_info[candidate_info['IAU_stripped'].isin(seth_unabsorbed_table51)]

,# Name,RA,Dec,ObsID,Counts,Z,Model,Cstat,nH,nH error plus,...,Gemini South alt,Gemini South best day,SDSS Plate,SDSS Fiber,SDSS MJD,Chandra Cycle,Chandra Exposure (ks),Count rate (/s),Chandra offset (deg),IAU_stripped
1,2CXO J081013.0+345137,122.554091,34.860185,10359,359,0.082851,main,662.361544,0.000784,0.023346,...,24.969802,2024-02-19 03:00:00.000,892,58,52378,10,7.85,0.045732,0.122921,J081013.0+345137
2,2CXO J123038.9+401614,187.662315,40.270759,10739,110,0.132453,main,542.677172,0.004464,0.281880,...,19.615924,2023-04-26 03:00:00.000,1986,599,53475,10,4.97,0.022133,0.116206,J123038.9+401614
4,2CXO J233731.7+002600,354.382457,0.433293,11728d,112,0.313821,main,541.416682,0.008832,NaN,...,59.193266,2023-10-12 03:00:00.000,682,493,52525,11,16.81,0.006663,0.161041,J233731.7+002600
10,2CXO J134736.4+173404,206.901667,17.567940,13232,116,0.044726,main,518.531046,0.032615,NaN,...,42.306693,2023-05-15 03:00:00.000,2742,442,54233,12,5.00,0.023200,0.141397,J134736.4+173404
11,2CXO J150117.0+014958,225.321194,1.832858,13247,91,0.128888,main,547.647372,0.000740,0.049035,...,58.019239,2023-06-03 03:00:00.000,539,435,52017,12,35.76,0.002545,0.097436,J150117.0+014958
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,2CXO J150117.0+014958,225.321270,1.832829,13246,240,0.129253,res,838.927465,0.001574,0.026841,...,58.019269,2023-06-03 03:00:00.000,539,435,52017,12,45.02,0.005331,0.097436,J150117.0+014958
161,2CXO J150117.0+014958,225.321270,1.832829,13255,89,0.129253,res,584.091208,0.003675,0.204252,...,58.019269,2023-06-03 03:00:00.000,539,435,52017,12,43.40,0.002051,0.097436,J150117.0+014958
164,2CXO J143232.4+340624,218.135420,34.107050,18475b,91,0.042640,res,477.783626,0.000000,0.016308,...,25.753779,2023-05-27 03:00:00.000,1645,251,53172,17,24.72,0.003681,0.071670,J143232.4+340624
171,2CXO J235720.1-005829,359.333890,-0.974996,6128,193,0.339302,res,568.004390,0.000000,0.005516,...,60.600860,2023-10-17 03:00:00.000,684,5,52523,DDT,17.93,0.010764,0.111952,J235720.1-005829


In [60]:
#Check koulouridis objects
koulouridis=pd.read_csv('/Users/kciurleo/Documents/kciurleo/AGN/csvs/koulouridis.csv')

koulouridis[['RA','Dec']].to_csv('/Users/kciurleo/Documents/kciurleo/AGN/csvs/koulouridis_coordinates.csv', index=False)